In [1]:
# 1. Environment Setup
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
# 2. Imports
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Milvus
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [4]:
# 3. Load and Split LangSmith Docs
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = splitter.split_documents(docs)

In [5]:
# 4. Store in Milvus Vector Store
vectordb = Milvus.from_documents(
    documents,
    OpenAIEmbeddings(),
    connection_args={"host": "localhost", "port": "19530"},
    collection_name="langsmith_docs"
)
retriever = vectordb.as_retriever()

In [6]:
# 5. LangSmith Retriever Tool
retriever_tool = create_retriever_tool(
    retriever,
    name="langsmith_search",
    description="Use this tool to search LangSmith documentation. Always use it for LangSmith-related questions."
)

In [24]:
retriever_tool.name

'langsmith_search'

In [7]:
# 6. Wikipedia Tool
wiki_tool = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
)

In [8]:
# 7. Arxiv Tool
arxiv_tool = ArxivQueryRun(
    api_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
)

In [9]:
# 8. Unified Tool List
tools = [retriever_tool, wiki_tool, arxiv_tool]

In [25]:
tools

[Tool(name='langsmith_search', description='Use this tool to search LangSmith documentation. Always use it for LangSmith-related questions.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000203FEC80E00>, retriever=VectorStoreRetriever(tags=['Milvus', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.milvus.Milvus object at 0x00000203FF5B5FA0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x00000203FEC80F40>, retriever=VectorStoreRetriever(tags=['Milvus', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.milvus.Milvus object at 0x00000203FF5B5FA0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}

In [36]:
# 9. Custom Prompt for RAG Tools Agent
# prompt = ChatPromptTemplate.from_messages([
#     SystemMessage(
#         content=(
#             "You are a helpful assistant that uses tools to retrieve information.\n"
#             "Always try to use the tools when the input is about LangSmith, Wikipedia topics, or academic papers.\n"
#             "Use retrieved information to answer. If unsure, say 'I don't know'."
#         )
#     ),
#     HumanMessage(content="{input}"),
#     MessagesPlaceholder(variable_name="agent_scratchpad")
# ])




prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful assistant that uses tools to retrieve information. "
     "Always try to use the tools when the input is about LangSmith, Wikipedia topics, or academic papers. "
     "Use retrieved information to answer. If unsure, say 'I don't know'."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [ ]:
# from langchain import hub
# # Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.messages

c:\Users\ankit\OneDrive\Desktop\GenRPT-On-Boarding\LangChain\langchain-env\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [37]:
# 10. Create the LLM Agent
llm = ChatOpenAI(model="gpt-4", temperature=0)

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [38]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typin

In [39]:
# 11. Run Agent Queries
response1 = agent_executor.invoke({"input": "Tell me about LangSmith"})
print("\nLangSmith Response:\n", response1)



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith'}`


LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performance and get human feedback on your data.Prompt EngineeringIterate on prompts, with automatic version control and collaboration features.
LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​

Get started with Lang

In [40]:
response2 = agent_executor.invoke({"input": "What's the paper 1605.08386 about?"})
print("\nArxiv Paper Response:\n", response2)



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe paper titled "Heat-bath random walks with Markov bases" was authored by Caprice Stanley and Tobias Windisch. It was published on May 26, 2016. The paper studies graphs on lattice points whose edges come from a finite set.

> Finished chain.

Arxiv Paper Response:
 {'input': "What's the paper 1605.08386 about?", 'output': 'The paper titled "Heat-bath random walks with Markov bases" was authored by Caprice Stanley and Tobias Windisch. It was published on May 26, 2016. The paper studies graphs on lattice points whose edges come from a finite set.'}


In [41]:
response3 = agent_executor.invoke({"input": "Tell me about the Eiffel Tower"})
print("\nWikipedia Response:\n", response3)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Eiffel Tower'}`


Page: Eiffel Tower
Summary: The Eiffel Tower (  EYE-fəl; French: Tour Eiffel [tuʁ ɛfɛl] ) is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave The Eiffel Tower is a wrought-iron lattice tower located on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower.

> Finished chain.

Wikipedia Response:
 {'input': 'Tell me about the Eiffel Tower', 'output': 'The Eiffel Tower is a wrought-iron lattice tower located on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower.'}
